In [1]:
from tensorflow import keras
import numpy as np
import math
import os
from tensorflow import keras
import librosa.display
import librosa
import matplotlib.pyplot as plt

In [2]:
DATASET_PATH ='Dataset/genres_original/'
SAMPLE_RATE = 22050
DURATION = 30
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION 

In [3]:
def save_mfcc(dataset_path  , n_mfcc = 13 , n_fft = 2048 , hop_length = 512 , num_segments = 5):
    data = {
        'mapping' :[] ,
        'mfcc' :[] ,
        'labels' : []
    }
    num_sample_per_segment = int(SAMPLES_PER_TRACK / num_segments) #so here every 30 second we would split then into 5 segments
    expected_num_mfcc_vector_per_segment = math.ceil(num_sample_per_segment / hop_length)
    print(f'{expected_num_mfcc_vector_per_segment} that is the length of the sequence')
    
    
    for i , (dirpath , dirnames , filenames) in enumerate(os.walk(dataset_path)):
        
        
        if dirpath != dataset_path:
            semantic_label = dirpath.split('/')[-1]
            data['mapping'].append(semantic_label)
            for file in filenames:
                file_path = os.path.join(dirpath , file)
                try :
                    signal , sr = librosa.load(file_path , sr = SAMPLE_RATE)
                    for s in range(num_segments):
                        start_sample = num_sample_per_segment * s
                        finish_sample = start_sample + num_sample_per_segment
                        mfcc = librosa.feature.mfcc(y = signal[start_sample:finish_sample] , sr = SAMPLE_RATE , 
                                                   n_mfcc=13 , n_fft = n_fft , hop_length = hop_length)
                        mfcc = mfcc.T # as we have the data (sequence number , number of features)
                        if len(mfcc) == expected_num_mfcc_vector_per_segment:
                            data['mfcc'].append(mfcc.tolist())
                            data['labels'].append(i-1)
                except:
                    pass
                        
            print(f"{dirpath.split('/')[-1]} is loaded successfully")
                        
    return data

In [4]:
data_dict = save_mfcc(DATASET_PATH  , num_segments=10)

130 that is the length of the sequence
blues is loaded successfully
classical is loaded successfully
country is loaded successfully
disco is loaded successfully
hiphop is loaded successfully
jazz is loaded successfully
jazz\.ipynb_checkpoints is loaded successfully
metal is loaded successfully
pop is loaded successfully
reggae is loaded successfully
rock is loaded successfully


In [5]:
data = np.array(data_dict['mfcc'])
label = np.array(data_dict['labels']).reshape(-1 , 1)

In [6]:
data.shape

(9996, 130, 13)

In [7]:
label.shape

(9996, 1)

In [8]:
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test =train_test_split(data , label  ,test_size=0.2 ,shuffle= True)
X_train, X_valid, y_train, y_valid =train_test_split(X_train_val , y_train_val  ,test_size=0.2)

In [9]:
model = keras.models.Sequential([
    
    keras.layers.LSTM(128, return_sequences=True, input_shape=[None, 13]),
    keras.layers.LayerNormalization(),
    keras.layers.LSTM(64, return_sequences=True),
    keras.layers.LayerNormalization(),
    keras.layers.LSTM(32),
    keras.layers.Dense(11 ,  activation="softmax")
])

model.compile(loss ='sparse_categorical_crossentropy', optimizer = keras.optimizers.Adam() , metrics = ['accuracy'])

In [10]:
model.fit(X_train , y_train ,validation_data=(X_valid , y_valid) , epochs=30 , batch_size=8)

Epoch 1/30
800/800 [==============================] - 94s 118ms/step - loss: 1.6805 - accuracy: 0.4023 - val_loss: 1.4765 - val_accuracy: 0.4644
Epoch 2/30
800/800 [==============================] - 92s 115ms/step - loss: 1.4140 - accuracy: 0.4892 - val_loss: 1.4689 - val_accuracy: 0.4500
Epoch 3/30
800/800 [==============================] - 92s 115ms/step - loss: 1.3216 - accuracy: 0.5184 - val_loss: 1.3162 - val_accuracy: 0.5194
Epoch 4/30
800/800 [==============================] - 92s 115ms/step - loss: 1.2571 - accuracy: 0.5478 - val_loss: 1.3480 - val_accuracy: 0.5031
Epoch 5/30
800/800 [==============================] - 92s 115ms/step - loss: 1.1934 - accuracy: 0.5705 - val_loss: 1.3086 - val_accuracy: 0.5319
Epoch 6/30
800/800 [==============================] - 92s 115ms/step - loss: 1.1345 - accuracy: 0.5987 - val_loss: 1.2168 - val_accuracy: 0.5706
Epoch 7/30
800/800 [==============================] - 92s 115ms/step - loss: 1.0725 - accuracy: 0.6201 - val_loss: 1.1632 - val_ac

In [11]:
model.evaluate(X_test ,y_test)

63/63 [==============================] - 3s 55ms/step - loss: 0.6615 - accuracy: 0.7965


[0.6615481376647949, 0.796500027179718]